In [4]:
import traceback
import re
from openai import OpenAI
import os

# Replace with your Groq API key
client = OpenAI(
    api_key="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", 
    base_url="https://api.groq.com/openai/v1"
)

def clean_code_block(code):
    # Remove triple backticks and optional 'python' identifier
    code = re.sub(r'^```(?:python)?\s*\n([\s\S]*?)\n```$', r'\1', code, flags=re.MULTILINE)
    return code.strip()

def generate_code(task, history=""):
    prompt = f"""
You are a Python coding assistant.
Write a complete and runnable Python function or script for the following task.

TASK:
{task}

PREVIOUS ATTEMPT (if any):
{history}

Only return the Python code inside a single Python code block.
"""
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful Python coding agent."},
            {"role": "user", "content": prompt}
        ]
    )
    code_raw = response.choices[0].message.content
    return clean_code_block(code_raw)

def run_code(code):
    try:
        local_vars = {}
        exec(code, {}, local_vars)
        return "Code ran successfully!", None
    except Exception:
        return None, traceback.format_exc()

def self_improving_agent():
    task = input("Enter the coding task: ")
    max_attempts = 5
    attempt = 0
    history = ""

    while attempt < max_attempts:
        print(f"\n🧠 Attempt {attempt + 1}")
        code = generate_code(task, history)
        print("\n📝 Generated Code:\n")
        print(code)

        result, error = run_code(code)
        if result:
            print("\n✅ Success:", result)
            break
        else:
            print("\n❌ Error:\n", error)
            history = f"The following code failed:\n{code}\nWith error:\n{error}"
            attempt += 1
    else:
        print("\n❗ Max attempts reached. Agent couldn't fix the issue.")

# 🔁 Run the agent
self_improving_agent()


🧠 Attempt 1

📝 Generated Code:

def sum4(nums):
    nums.sort()
    pairs = []
    for i in range(len(nums) - 3):
        if i > 0 and nums[i] == nums[i - 1]:
            continue
        l, r = i + 1, len(nums) - 1
        while l < r:
            s = nums[i] + nums[l] + nums[r]
            if s < 4:
                l += 1
            elif s > 4:
                r -= 1
            else:
                pairs.append((nums[i], nums[l], nums[r]))
                while l < r and nums[l] == nums[l + 1]:
                    l += 1
                while l < r and nums[r] == nums[r - 1]:
                    r -= 1
                l += 1; r -= 1
    return [[x, y, z] for x, y, z in set(tuple(sorted(p)) for p in pairs)]

✅ Success: Code ran successfully!
